# Optimización de Particiones con Z-Order y Manejo de Skew

**Objetivos del Laboratorio:**

Uso eficiente de particiones para distribuir datos en grandes volúmenes.

Optimización de consultas con Z-ordering para mejorar la búsqueda en múltiples columnas.

Técnicas para manejar el skew (desbalance) en particiones.

Monitoreo de rendimiento sin depender de Spark UI.

## Paso 1: Cargar el Dataset

In [0]:
# Montar el contenedor de Azure Data Lake Storage (ADLS)
dbutils.fs.mount(
    source = "",
    mount_point = "/mnt/",
    extra_configs = {"": ""}
)



In [0]:

# Cargar el dataset desde Azure Data Lake Storage
df = spark.read.csv("/mnt/datalake/retail_dataset.csv", header=True, inferSchema=True)

# Mostrar las primeras filas del dataset
df.show(5)

+---+----------+-----------+--------------+------------+-----+--------+------------+
|_c0|      Date|Customer_ID|Transaction_ID|SKU_Category|  SKU|Quantity|Sales_Amount|
+---+----------+-----------+--------------+------------+-----+--------+------------+
|  1|2016-01-02|       2547|             1|         X52|0EM7L|     1.0|        3.13|
|  2|2016-01-02|        822|             2|         2ML|68BRQ|     1.0|        5.46|
|  3|2016-01-02|       3686|             3|         0H2|CZUZX|     1.0|        6.35|
|  4|2016-01-02|       3719|             4|         0H2|549KK|     1.0|        5.59|
|  5|2016-01-02|       9200|             5|         0H2|K8EHH|     1.0|        6.88|
+---+----------+-----------+--------------+------------+-----+--------+------------+
only showing top 5 rows



## Paso 2: Reparticionamiento Basado en Cardinalidad

Comenzamos con un reparticionamiento avanzado basado en columnas de cardinalidad alta para distribuir eficientemente los datos a lo largo de las particiones.

## 2.1. Evaluación de la cardinalidad de columnas clave

Primero evaluamos la cardinalidad de las columnas candidatas para particionamiento. Idealmente, queremos columnas que tengan suficientes valores únicos para evitar particiones desbalanceadas.

In [0]:
# Evaluar la cardinalidad de las columnas clave
df.select("Customer_ID", "SKU_Category", "Date").groupBy("Customer_ID", "SKU_Category", "Date").count().show()


+-----------+------------+----------+-----+
|Customer_ID|SKU_Category|      Date|count|
+-----------+------------+----------+-----+
|       2237|         LDZ|2016-01-04|    1|
|        754|         TW8|2016-01-04|    1|
|       1082|         1VL|2016-01-04|    1|
|        853|         BZU|2016-01-07|    1|
|       7223|         R6E|2016-01-07|    1|
|       3827|         JPI|2016-01-08|    1|
|       3957|         P42|2016-01-08|    1|
|       2436|         P42|2016-01-08|    1|
|       5253|         29A|2016-01-08|    1|
|       3719|         U5F|2016-01-11|    1|
|       3719|         0H2|2016-01-11|    1|
|       2811|         IEV|2016-01-11|    1|
|       2402|         IEV|2016-01-11|    1|
|        199|         U5F|2016-01-11|    1|
|       7060|         MOE|2016-01-11|    1|
|       1591|         1L6|2016-01-12|    1|
|         70|         A38|2016-01-13|    1|
|       3182|         LPF|2016-01-13|    1|
|       9095|         R6E|2016-01-13|    1|
|       5612|         RU6|2016-0

## 2.2. Reparticionamiento basado en columnas de cardinalidad alta

Una vez identificadas las columnas de alta cardinalidad, aplicamos reparticionamiento. Aquí particionaremos por Customer_ID y Date, que suelen ser columnas con alta cardinalidad en datasets transaccionales.

In [0]:
# Reparticionamiento eficiente del dataset
df_repartitioned = df.repartition("Customer_ID", "Date")

# Guardar el dataset reparticionado en formato Delta
df_repartitioned.write.partitionBy("Customer_ID", "Date").format("delta").mode("overwrite").save("/mnt/datalake/partitioned_data_delta")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

**Optimización del Código:**
Reducir el Número de Particiones de Reparticionamiento: El número de particiones por defecto en Spark podría ser demasiado alto para datasets pequeños o medianos, lo que puede resultar en un procesamiento ineficiente. Puedes ajustar el número de particiones con la configuración spark.sql.shuffle.partitions:

In [0]:
# Ajustar el número de particiones de shuffle (reducirlo si el dataset no es masivo)
spark.conf.set("spark.sql.shuffle.partitions", "200")  # Ajusta este valor según el tamaño del dataset


**Evitar la Duplicación de Particionamiento:** Dado que estás reparticionando primero con .repartition("Customer_ID", "Date") y luego guardando el dataset usando partitionBy("Customer_ID", "Date"), estás aplicando una doble partición. Si ya particionas en el guardado, no necesitas hacerlo antes con repartition().

Optimización: Elimina el repartition() y usa solo partitionBy() al guardar.

In [0]:
# Guardar el dataset directamente en formato Delta con particiones
df.write.partitionBy("Customer_ID", "Date").format("delta").mode("overwrite").save("/mnt/datalake/partitioned_data_delta")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

**Usar Coalesce para Minimizar el Número de Particiones:** Si el reparticionamiento es necesario y tu dataset tiene demasiadas particiones, puedes usar .coalesce() para reducir el número de particiones antes de guardar el archivo, lo que minimiza el tiempo de escritura al disco.

In [0]:
# Reparticionar solo si es absolutamente necesario, y luego reducir el número de particiones
df_repartitioned = df.repartition("Customer_ID", "Date").coalesce(200)  # Ajusta el valor según el tamaño

# Guardar el dataset en formato Delta
df_repartitioned.write.partitionBy("Customer_ID", "Date").format("delta").mode("overwrite").save("/mnt/datalake/partitioned_data_delta")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

**Especificar write.option("overwriteSchema", "true"):** Para evitar potenciales problemas con el esquema, puedes incluir la opción overwriteSchema para reducir el tiempo si el esquema se mantiene constante en cada escritura:

In [0]:
df.write.partitionBy("Customer_ID", "Date").format("delta").mode("overwrite").option("overwriteSchema", "true").save("/mnt/datalake/partitioned_data_delta")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

**Caché para Consultas Frecuentes:** Si el dataset se usa en varias operaciones después de este proceso de particionamiento, puedes almacenar en caché el dataframe reparticionado para evitar operaciones repetitivas costosas:

In [0]:
df_repartitioned = df.repartition("Customer_ID", "Date").cache()

# Guardar el dataset en formato Delta
df_repartitioned.write.partitionBy("Customer_ID", "Date").format("delta").mode("overwrite").save("/mnt/datalake/partitioned_data_delta")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# Optimización sin doble reparticionamiento
spark.conf.set("spark.sql.shuffle.partitions", "200")  # Ajusta el número según el tamaño del dataset

# Guardar el dataset en formato Delta directamente con particiones
df.write.partitionBy("Customer_ID", "Date").format("delta").mode("overwrite").option("overwriteSchema", "true").save("/mnt/datalake/partitioned_data_delta")


## Paso 3: Optimización con Z-Order

### 3.1. Aplicar Z-Order para mejorar consultas

Z-Order es una técnica que permite mejorar el rendimiento de consultas cuando se filtra en múltiples columnas, optimizando el almacenamiento de datos para que los valores de columnas filtradas frecuentemente estén cerca unos de otros en el disco.

In [0]:
# Leer el dataset en formato Delta
df_delta = spark.read.format("delta").load("/mnt/datalake/partitioned_data_delta")

# Aplicar Z-Order sobre las columnas más consultadas
df_delta.write.format("delta").mode("overwrite").option("zorder", "SKU_Category").save("/mnt/datalake/z_ordered_data")


**Explicación:**

Z-Order mejora el rendimiento cuando las consultas frecuentemente filtran por Customer_ID, Date y SKU_Category. Al aplicarlo, los datos se reordenan en disco para facilitar las búsquedas rápidas y minimizar la cantidad de datos escaneados.

## Paso 4: Manejo de Skew (Desbalance de Datos)

## 4.1. Identificación del skew en particiones

El skew ocurre cuando algunas particiones contienen significativamente más datos que otras. Esto puede ralentizar las consultas ya que algunas tareas tendrán más trabajo que otras. Para identificar skew, se puede analizar el tamaño de las particiones después del reparticionamiento.

In [0]:
# Contar filas por partición para detectar skew
df_repartitioned.groupBy(spark_partition_id()).count().show()


## 4.2. Soluciones para el skew

Si se detecta skew, se pueden aplicar varias técnicas para corregirlo:

**Salting:** Añadir una columna "salting" para dividir las particiones más grandes.

In [0]:
from pyspark.sql.functions import col, expr

# Crear una columna "salt" para distribuir mejor las particiones
df_salted = df.withColumn("salt", expr("floor(rand() * 10)"))

# Reparticionar utilizando "salt" para corregir el skew
df_salted_repartitioned = df_salted.repartition("salt", "Customer_ID", "Date")

# Guardar el dataset corregido por skew
df_salted_repartitioned.write.partitionBy("salt", "Customer_ID", "Date").format("delta").mode("overwrite").save("/mnt/datalake/salted_partitioned_data")


**Explicación:**

Salting distribuye mejor las filas en las particiones al añadir una nueva columna salt, lo que ayuda a evitar que una sola partición contenga demasiados datos.

## Paso 5: Monitoreo del Rendimiento

## 5.1. Medir el tiempo de ejecución de las consultas optimizadas

Monitoreamos el tiempo de ejecución para asegurarnos de que las optimizaciones (Z-Order, particionamiento, y corrección de skew) están mejorando el rendimiento.

In [0]:
import time

# Medir el tiempo de ejecución de una consulta optimizada
start_time = time.time()

# Consulta típica sobre el dataset optimizado
df_delta.filter("Customer_ID = 12345").filter("Date >= '2024-01-01'").filter("SKU_Category = 'Electronics'").groupBy("SKU").sum("Sales_Amount").show()

print(f"Tiempo de ejecución: {time.time() - start_time} segundos")


## 5.2. Monitoreo de recursos

Monitoreamos el uso de recursos durante la ejecución, como memoria y CPU, para verificar si las optimizaciones están haciendo un uso eficiente de los recursos del clúster.

In [0]:
# Monitorear recursos durante la ejecución
executor_info = spark.sparkContext.statusTracker().getExecutorInfos()
for executor in executor_info:
    print(f"ID del ejecutor: {executor.executorId}, Uso de memoria: {executor.memoryUsed}, Núcleos totales: {executor.totalCores}")


## Paso 6: Limpieza de Particiones Antiguas

Es una buena práctica limpiar particiones antiguas que ya no sean necesarias para evitar el almacenamiento innecesario y mejorar el rendimiento.

In [0]:
# Limpieza de particiones no usadas en un dataset Delta
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/mnt/datalake/z_ordered_data")
delta_table.vacuum(retentionHours=168)  # Retención de 7 días
